In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [3]:
import pandas as pd
realDF = pd.read_csv('../data-processing/genuine_accounts_processed.csv')
ss1DF = pd.read_csv('../data-processing/social_spambots_1_processed.csv')
ss2DF = pd.read_csv('../data-processing/social_spambots_2_processed.csv')
ss3DF = pd.read_csv('../data-processing/social_spambots_3_processed.csv')
ffDF = pd.read_csv('../data-processing/fake_followers_processed.csv')
ts1DF = pd.read_csv('../data-processing/traditional_spambots_1_processed.csv')


In [4]:
def data_split (df, split=0.7):
    train_num = int(len(df.index)*split)
    return df.iloc[:train_num,:], df.iloc[train_num+1:,:]

realTrain, realTest = data_split(realDF)
ss1Train, ss1Test = data_split(ss1DF)
ss2Train, ss2Test = data_split(ss2DF)
ss3Train, ss3Test = data_split(ss3DF)
ts1Train, ts1Test = data_split(ts1DF)
ffTrain, ffTest = data_split(ffDF)

In [6]:
def trainRF (reals, bots, n_estimators=100):

    data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
    target = np.array([0] * sum(len(real.index) for real in reals) + [1] * sum(len(bot.index) for bot in bots))
    rfc = RandomForestClassifier(n_estimators=n_estimators)
    rfc = rfc.fit(data, target)
    return rfc

In [9]:
def testRF(reals, bots, RF):
    data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
    target = np.array([0] * sum(len(real.index) for real in reals) + [1] * sum(len(bot.index) for bot in bots))
    return RF.score(data, target)

In [18]:
def testRF_ESC(reals, bots, RFs):
    data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
    target = np.array([0] * sum(len(real.index) for real in reals) + [1] * sum(len(bot.index) for bot in bots))
    totalAcc = 0
    for i in range(data.shape[0]):
        totalAcc += 1.0-abs(max(float(RF.predict([data[i]])[0]) for RF in RFs)-float(target[i]))
    return totalAcc/data.shape[0]

In [7]:
RF = []
RF.append(trainRF([realTrain], [ss1Train, ss2Train, ss3Train, ffTrain, ts1Train]))
RF.append(trainRF([realTrain], [ss1Train, ss2Train, ss3Train]))
RF.append(trainRF([realTrain], [ts1Train]))
RF.append(trainRF([realTrain], [ffTrain]))

C:\Users\56830\AppData\Local\Temp/ipykernel_51344/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
C:\Users\56830\AppData\Local\Temp/ipykernel_51344/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
C:\Users\56830\AppData\Local\Temp/ipykernel_51344/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()
C:\Users\56830\AppData\Local\Temp/ipykernel_51344/2087584835.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be

In [10]:
print(testRF([realTest], [ss1Test, ss2Test, ss3Test, ffTest, ts1Test], RF[0]))

0.9897879025923016


C:\Users\56830\AppData\Local\Temp/ipykernel_51344/1317824033.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()


In [17]:
print(testRF_ESC([realTest], [ss1Test, ss2Test, ss3Test, ts1Test, ffTest], RF))

C:\Users\56830\AppData\Local\Temp/ipykernel_51344/706939187.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat(reals+bots).fillna(0).drop('Unnamed: 0', 1).to_numpy()


0.9903115998952605
